## 调整向下生成方法

In [15]:
from btree import *

### 增加判叶属性

In [ ]:
Node.is_leaf = lambda self:self.left is None and self.right is None

In [ ]:
t = BTree(2)
print(t.is_leaf())
t.left = Node(3)
print(t.is_leaf())

In [ ]:
class rex():
    def __init__(self):
        self.a = 1
    pass
rex.new_method = lambda self:self.a

In [ ]:
obj = rex()
obj.new_method()

### 变量作用域测试

1. 将函数导入本地空间后，定义空间仍在模块中，只是调用更直接了
2. 使用 from import 与直接 import ，二者使用同一模块空间

In [4]:
from test import *

In [14]:
import test
test.glvalue

1

In [13]:
zong()

1


In [10]:
glvalue = 3

### 修改 tree_to_list

In [17]:
tree = BTree.random_binary_tree(5)
print(tree)


     ___0___
    /       \
  _0        _0__________________
 /  \      /                    \
10   27   20             ________0__
                        /           \
                    ___0___          0
                   /       \        / \
                 _0        _0      6   25
                /  \      /  \
               28   22   10   25



In [19]:
tree.max_leaf_depth

5

In [16]:
def node_to_list(tree,depth):
    """节点树 -> 列表"""
    positions = [None for i in range(2**(depth+1)-1)]
    for node in tree:
        # 位置平移公式
        dep = node.depth - tree.depth
        pos = node.position - (tree.position-1)*(2**dep)
        positions[pos-1]=node.value
    return positions

### 修改函数

In [ ]:
def next_level(tree,ak):
    """选取 ak 个位置，生成下一层树"""
    # 三类节点
    news = [] # 新一层树
    sep,not_sep,whatever = [],[],[]
    for node in tree.last_layer:
        node_pos = node.position
        if node_pos in nonleaf_cost: # 非叶子节点
            sep.append(node_pos)
        elif node_pos in leaf_cost: # 带开销的叶子节点
            not_sep.append(node_pos)
            node.value = leaf_cost[node_pos]
        else: # 零开销位置
            whatever.append(node_pos)
    # 产生新树
    if ak < len(sep): # 展开少，取 sep 子集，增加未取部分开销
        for choice in choose(sep,ak):
            new = tree.new_tree_by_list(choice)
            new.cost += sum(nonleaf_cost[i] for i in sep if i not in choice)
            news.append(new)
    elif len(sep) <= ak <= len(sep)+len(whatever): # 展开适中，不增加开销
        choice = sep + whatever[:ak-len(sep)]
        news = [tree.new_tree_by_list(choice)]
    else: # 展开多，取 not_sep 子集，增加选取部分开销
        for choice in choose(not_sep,ak-len(sep)-len(whatever)):
            new = tree.new_tree_by_list(sep+whatever+choice)
            new.cost += sum(leaf_cost[i] for i in choice)
            news.append(new)
    if nonleaves[k+1]==0:
        is_end = True
        if k+2<n: # 剩下节点合并
            for new in news:
                new.cost += sum(nonleaf_cost.get(node.position,0) for node in new.last_layer)
    return news,is_end

In [ ]:
def main(positions,leaves):
    """函数打包"""
    n = len(leaves)
    old = BTree.list_to_tree(positions)
    nonleaf_cost,leaf_cost = get_nodes_cost(old) # 节点开销信息
    nonleaves = leaves_to_nonleaves(leaves)
    root = BTree(0)
    root.cost = 0 # 初始开销
    min_cost = sum(leaf_cost.values()) # 最小开销
    optimal = None # 最优解
    tmp_trees = [root] # 待遍历集合
    while len(tmp_trees):
        tree = tmp_trees[0] # 取最小开销树
        tmp_trees = tmp_trees[1:]
        news,is_end = next_level(tree,nonleaves,nonleaf_cost,leaf_cost)
        # 处理新树
        if is_end: # 树已完全展开
            for new in news:
                if new.cost < min_cost:
                    min_cost = new.cost
                    optimal = new
                    tmp_trees = [tree for tree in tmp_trees if tree.cost<= min_cost]
        else: # 树不完整
            tmp_trees.extend(news) # 加入新结果
            tmp_trees.sort(key=lambda x:x.cost-x.depth) # 排序
    return optimal,min_cost